# **Maestría en Inteligencia Artificial Aplicada** 

## **Curso: Inteligencia Artificial y Aprendizaje Automático**

### Tecnológico de Monterrey

### Prof Luis Eduardo Falcón Morales

## **Adtividad de la Semana 7**

### **Red Neuronal Artificial - Perceptrón Multicapa : Multilayer Perceptrón (MLP)**


**Nombres y matrículas de los integrantes del equipo:**

*   
*   
*   
*   
*   


En cada sección deberás incluir todas las líneas de código necesarias para responder a cada uno de los ejercicios.

Esta Tarea se deberá resolver en equipos, de acuerdo a como fueron integrados al inicio del curso.

El artículo del 2016 de Moro, Rita & Vala relacionado al presente problema lo encuentras aquí: 
https://www.semanticscholar.org/paper/Predicting-social-media-performance-metrics-and-of-Moro-Rita/dec55692590820754b53c916e29bb2b42c0e5104  
 
Y el conjunto de datos utilizado es el de la siguiente liga de la UCI: 
https://archive.ics.uci.edu/ml/datasets/Facebook+metrics 

El archivo se llama dataset_Facebook.csv y contiene 500 registros y 19 columnas. 
El objetivo es determinar el impacto generado por un anuncio de una marca de cosméticos en 
Facebook, en el cual se intentaron varias variantes en la manera de mostrar el anuncio. Siguiendo el 
artículo, deberás considerar como predictores o variables de entrada aquellas que se indican en la Tabla 
3. Por otro lado, en la Tabla 2, los autores consideran varios casos para la variable de salida, intentando 
determinar cuál puede ser el mejor caso para medir el éxito de la campaña.  

In [64]:
# Incluye aquí todos módulos, librerías y paquetes que requieras.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.model_selection import cross_validate, learning_curve, validation_curve
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix, make_scorer
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import RepeatedKFold
from sklearn import metrics
from math import isnan
from sklearn.preprocessing import FunctionTransformer, PowerTransformer
from sklearn.preprocessing import power_transform

# **Ejercicio-1.** 

En esta tarea considera únicamente la siguiente variable de salida que se concluye que es una de las mejores en el artículo antes citado: ‘Lifetime People who have liked a Page and engaged with a post'. Renombra dicha variable como “LPE” . Como variables de entrada selecciona las 7 variables que indican los autores en la Tabla 3 del artículo citado.

In [65]:
dataset_fb = pd.read_csv('./Facebook_metrics/dataset_Facebook.csv',sep=';')
dataset_fb.head()

,Page total likes,Type,Category,Post Month,Post Weekday,Post Hour,Paid,Lifetime Post Total Reach,Lifetime Post Total Impressions,Lifetime Engaged Users,Lifetime Post Consumers,Lifetime Post Consumptions,Lifetime Post Impressions by people who have liked your Page,Lifetime Post reach by people who like your Page,Lifetime People who have liked your Page and engaged with your post,comment,like,share,Total Interactions
0,139441,Photo,2,12,4,3,0.0,2752,5091,178,109,159,3078,1640,119,4,79.0,17.0,100
1,139441,Status,2,12,3,10,0.0,10460,19057,1457,1361,1674,11710,6112,1108,5,130.0,29.0,164
2,139441,Photo,3,12,3,3,0.0,2413,4373,177,113,154,2812,1503,132,0,66.0,14.0,80
3,139441,Photo,2,12,2,10,1.0,50128,87991,2211,790,1119,61027,32048,1386,58,1572.0,147.0,1777
4,139441,Photo,2,12,2,3,0.0,7244,13594,671,410,580,6228,3200,396,19,325.0,49.0,393


In [66]:
dataset_fb.shape

(500, 19)

In [67]:
dataset_fb = dataset_fb.dropna()
dataset_fb.shape

(495, 19)

In [68]:
dataset_articulo = dataset_fb.iloc[:,[0,1,2,3,4,5,6,10,14,16]].rename(columns = {'Lifetime People who have liked your Page and engaged with your post':'LPE'})
dataset_articulo.rename(columns = {'Lifetime Post Consumers':'LPC'}, inplace = True)
dataset_articulo.rename(columns = {'Page total likes':'PTL'}, inplace = True)
dataset_articulo.rename(columns = {'Post Month':'PM'}, inplace = True)
dataset_articulo.rename(columns = {'Post Weekday':'PW'}, inplace = True)
dataset_articulo.rename(columns = {'Post Hour':'PH'}, inplace = True)
dataset_articulo.Paid =  dataset_articulo.Paid.astype('int')
dataset_articulo.head()

,PTL,Type,Category,PM,PW,PH,Paid,LPC,LPE,like
0,139441,Photo,2,12,4,3,0,109,119,79.0
1,139441,Status,2,12,3,10,0,1361,1108,130.0
2,139441,Photo,3,12,3,3,0,113,132,66.0
3,139441,Photo,2,12,2,10,1,790,1386,1572.0
4,139441,Photo,2,12,2,3,0,410,396,325.0


In [69]:
X = dataset_articulo.iloc[:,: -3]
X.head()

,PTL,Type,Category,PM,PW,PH,Paid
0,139441,Photo,2,12,4,3,0
1,139441,Status,2,12,3,10,0
2,139441,Photo,3,12,3,3,0
3,139441,Photo,2,12,2,10,1
4,139441,Photo,2,12,2,3,0


In [70]:
Y_like = dataset_articulo.iloc[:, -1]
Y_LPE = dataset_articulo.iloc[:, -2]
Y_LPC = dataset_articulo.iloc[:, -3]

# **Ejercicio-2.**
Realiza una partición de los datos con 100 datos de Prueba y el resto para entrenamiento y 
validación.  

In [71]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y_LPE, test_size=100, random_state=55
)

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)


(395, 7)
(100, 7)
(395,)
(100,)


# **Ejercicio-3.**
Definirás tus propias funciones de errores para este problema de regresión. Los errores que 
utilizarás son la raíz cuadrada del error cuadrático medio RMSE, el error absoluto medio MAE y el 
error porcentual absoluto medio MAPE. 

In [72]:
from ast import Sub


def mi_RMSE(actual,predicted):
    MSE=np.mean(np.power(np.subtract(actual,predicted),2))
    return MSE**(1/2)

# Incluye aquí las líneas de código que definan a la función: mi_MAE:
def mi_MAE(actual,predicted):
    MAE=np.mean(np.absolute(np.subtract(actual,predicted)))
    return MAE


# Incluye aquí las líneas de código que definan a la función: mi_MAPE:
def mi_MAPE(actual,predicted):
    return (np.mean(np.absolute(np.divide(np.subtract(actual,predicted),actual))) * 100)

# **Ejercicio-4.**
En la página de la UCI, así como en el artículo de los autores previamente citado encuentras 
información en relación al significado de cada variable. Haz una análisis de tus datos y lleva a cabo 
las transformaciones que consideres adecuadas tanto en los datos de entrada, como en las de salida. 
Utiliza un Pipeline para evitar el filtrado de información. 

In [73]:
round(dataset_articulo.describe(),1)

,PTL,Category,PM,PW,PH,Paid,LPC,LPE,like
count,495.0,495.0,495.0,495.0,495.0,495.0,495.0,495.0,495.0
mean,123173.3,1.9,7.0,4.1,7.8,0.3,804.2,614.1,179.1
std,16203.8,0.9,3.3,2.0,4.4,0.4,885.2,614.3,324.4
min,81370.0,1.0,1.0,1.0,1.0,0.0,9.0,9.0,0.0
25%,112324.0,1.0,4.0,2.0,3.0,0.0,335.0,297.5,57.0
50%,129600.0,2.0,7.0,4.0,9.0,0.0,555.0,416.0,101.0
75%,136393.0,3.0,10.0,6.0,11.0,1.0,969.0,658.5,188.0
max,139441.0,3.0,12.0,7.0,23.0,1.0,11328.0,4376.0,5172.0


In [74]:
datos_categoricos = [
    "Type",
    "Category",
    "PM",
    "PW",
    "PH",
]

datos_numericos = ["PTL"]
datos_binarios = ["Paid"]



pipeline_catogoricos = Pipeline(
    steps=[
        ("impModa", SimpleImputer(strategy="most_frequent")),
        ("OneHotE", OneHotEncoder(handle_unknown="ignore")),
    ]
)
pipelines_categoricos_nombre = datos_categoricos

pipeline_numericos = Pipeline(
        steps=[
        ("impMediana", SimpleImputer(strategy="median")),
        ("Box-Cox", PowerTransformer(method="box-cox")),
        ("escalaNum", MinMaxScaler()),
        ]
)

pipeline_numericos_nombre = datos_numericos
pipeline_binarios_nombre = datos_binarios

In [75]:
ColTrans = ColumnTransformer(
    transformers=[
        ("catpipe", pipeline_catogoricos, pipelines_categoricos_nombre + pipeline_binarios_nombre),
        ("numpipe", pipeline_numericos, pipeline_numericos_nombre),
    ],
    remainder="passthrough",
)


# **Ejercicio-5.**
Utiliza la función Dummy para modelos de regresión de scikit-learn con el conjunto que tienes de 
datos de entrenamiento y validación. Para ello particiónalos en 100 para validación y 300 para 
entrenamiento. Encuentra los errores RMSE, MAE y MAPE para los conjuntos de entrenamiento y 
validación. Estos serán tus errores máximos que deberás tomar como referencia en el resto de la 
actividad. Consulta su documentación correspondiente:  
https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyRegressor.html  
 

In [76]:
X_val_dummy , X_train_dummy , Y_val_dummy , Y_train_dummy = train_test_split(X_train, Y_train, train_size=100, random_state=55)

modelo_dummy = DummyRegressor(strategy="mean") # Default
modelo_dummy.fit(X_train_dummy, Y_train_dummy)

DummyRegressor()

In [77]:
yhat_train_dummy = modelo_dummy.predict(X_train_dummy)
yhat_val_dummy = modelo_dummy.predict(X_val_dummy)

In [78]:
mi_rmse = mi_RMSE(Y_train_dummy, yhat_train_dummy)
mi_mae = mi_MAE(Y_train_dummy, yhat_train_dummy)
mi_mape = mi_MAPE(Y_train_dummy, yhat_train_dummy)
print('ENTRENAMIENTO:')

print('mi_RMSE: ', round(mi_rmse,2))
print('mi_MAE: ', round(mi_mae,2))
print('mi_MAPE: ', round(mi_mape,2))

print('PRUEBA:')
mi_rmse = mi_RMSE(Y_val_dummy, yhat_val_dummy)
mi_mae = mi_MAE(Y_val_dummy, yhat_val_dummy)
mi_mape = mi_MAPE(Y_val_dummy, yhat_val_dummy)

print('mi_RMSE: ', round(mi_rmse,2))
print('mi_MAE: ', round(mi_mae,2))
print('mi_MAPE: ', round(mi_mape,2))

ENTRENAMIENTO:
mi_RMSE:  548.37
mi_MAE:  364.3
mi_MAPE:  126.25
PRUEBA:
mi_RMSE:  572.35
mi_MAE:  365.88
mi_MAPE:  114.93


# **Ejercicio-6.**
Usando los modelos de regresión lineal múltiple, el bosque aleatorio y el perceptrón multicapa con 
sus valores predeterminados, lleva a cabo su entrenamiento con repeticiones de validación cruzada 
(RepeatedKFold) y desplegando los errores RMSE, MAE y MAPE. Recuerda evitar el filtrado de 
información usando los datos que obtuviste en el ejercicio 2. Incluye las conclusiones sobre el mejor 
modelo encontrado en esta primera aproximación. En particular ¿hay alguno sobreentrenado o 
subentrenado? NOTA: Recuerda que puedes aumentar en dado caso el número máximo de 
iteraciones para que todos los modelos converjan.  

In [79]:
def crearModelos():
    return {
        "LR": LinearRegression(),
        "RF": RandomForestRegressor(n_estimators=100, random_state=55),
        "MLP": MLPRegressor(random_state=55,max_iter=5500),
    }

In [80]:
modelos = crearModelos()
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=55)
metricas = {
    "RMSE": make_scorer(mi_RMSE),
    "MAE": make_scorer(mi_MAE),
    "MAPE": make_scorer(mi_MAPE),
}

resultados = {}

for nombre, modelo in modelos.items():
    print(nombre)
    pipe = Pipeline(steps=[("preprocesamiento", ColTrans), ("modelo", modelo)])
    resultados[nombre] = cross_validate(
        pipe,
        X_train,
        Y_train,
        cv=cv,
        scoring=metricas,
        return_train_score=True,
        n_jobs=-1,
    )
    print('ENTRENAMIENTO:')
    print('mean RMSE: %.3f (%.4f)\nmean MAE: %.3f (%.4f)\nmean MAPE: %.3f (%.4f)\n' % (
                                                                          np.mean(resultados[nombre]['train_RMSE']),
                                                                          np.std(resultados[nombre]['train_RMSE']),
                                                                          np.mean(resultados[nombre]['train_MAE']),
                                                                          np.std(resultados[nombre]['train_MAE']), 
                                                                          np.mean(resultados[nombre]['train_MAPE']),
                                                                          np.std(resultados[nombre]['train_MAPE']),
                                                                          ))
    
    
    print('VALIDACION (interna al método de validación cruzada):')
    print('mean RMSE: %.3f (%.4f)\nmean MAE: %.3f (%.4f)\nmean MAPE: %.3f (%.4f)\n' % (
                                                                          np.mean(resultados[nombre]['test_RMSE']),
                                                                          np.std(resultados[nombre]['test_RMSE']),
                                                                          np.mean(resultados[nombre]['test_MAE']),
                                                                          np.std(resultados[nombre]['test_MAE']), 
                                                                          np.mean(resultados[nombre]['test_MAPE']),
                                                                          np.std(resultados[nombre]['test_MAPE']),
                                                                          ))
    

LR
ENTRENAMIENTO:
mean RMSE: 425.957 (12.2378)
mean MAE: 269.178 (6.0630)
mean MAPE: 85.414 (4.9390)

VALIDACION (interna al método de validación cruzada):
mean RMSE: 469.900 (107.5457)
mean MAE: 306.614 (39.7680)
mean MAPE: 98.094 (43.1163)

RF
ENTRENAMIENTO:
mean RMSE: 193.596 (8.9663)
mean MAE: 116.753 (4.0889)
mean MAPE: 34.601 (2.2596)

VALIDACION (interna al método de validación cruzada):
mean RMSE: 495.112 (118.7357)
mean MAE: 309.033 (51.0961)
mean MAPE: 92.406 (38.6488)

MLP
ENTRENAMIENTO:
mean RMSE: 358.959 (14.7820)
mean MAE: 222.431 (7.4976)
mean MAPE: 71.000 (4.6241)

VALIDACION (interna al método de validación cruzada):
mean RMSE: 467.650 (114.9028)
mean MAE: 302.560 (44.7316)
mean MAPE: 94.530 (43.3854)



#**Ejercicio-7.**

In [ ]:
None

#**Ejercicio-8.**

In [89]:
mi_modelo_MLP = MLPRegressor()
dicc_grid = {'hidden_layer_sizes': (100,), 'alpha': (0.0001, 0.001, 0.01, 0.1, 1, 10, 100),  'learning_rate_init': (0.0001, 0.001, 0.01, 0.1, 1, 10, 100), 'random_state' : (55,)}
grid = GridSearchCV(estimator=mi_modelo_MLP, param_grid=dicc_grid, cv=cv, scoring=make_scorer(mi_RMSE),n_jobs=-1)
Xx = ColTrans.fit_transform(X_train)
grid.fit(Xx, np.ravel(Y_train))

GridSearchCV(cv=RepeatedKFold(n_repeats=3, n_splits=10, random_state=55),
             estimator=MLPRegressor(), n_jobs=-1,
             param_grid={'alpha': (0.0001, 0.001, 0.01, 0.1, 1, 10, 100),
                         'hidden_layer_sizes': (100,),
                         'learning_rate_init': (0.0001, 0.001, 0.01, 0.1, 1, 10,
                                                100),
                         'random_state': (55,)},
             scoring=make_scorer(mi_RMSE))

In [91]:
mejor_modelo_MLPR = MLPRegressor(hidden_layer_sizes=grid.best_params_.get('hidden_layer_sizes'),
                                alpha=grid.best_params_.get('alpha'),
                                learning_rate_init=grid.best_params_.get('learning_rate_init'),
                                random_state=grid.best_params_.get('random_state'),
                                )
mejores_params_MLPR = grid.best_params_
pipe = Pipeline(steps=[("ct", ColTrans), ("m", mejor_modelo_MLPR)])
pipe.fit(X_train, Y_train)
predictions = pipe.predict(X_test)

#**Ejercicio-9.**

In [ ]:
None

# **Ejercicio-10.**
Repite el ejercicio 8 y 9 para el modelo de bosque aleatorio para buscar sus mejores hiperparámetros (realiza la búsqueda con aquellos hiperparámetros que consideres más adecuados) y usando el conjunto de Prueba. Y realiza igualmente el análisis de importancia de factores con este modelo con un diagrama de barras.

In [85]:
mi_modelo_RF = RandomForestRegressor()
dicc_grid = {'ccp_alpha':[0.0, 0.0001, 0.001, 0.01, 0.1],
             'criterion':["squared_error", "absolute_error", "poisson"],
             'max_depth':range(1,21,2),
             'min_samples_split':range(2, 12, 2),
            }

grid = GridSearchCV(estimator=mi_modelo_RF, param_grid=dicc_grid, cv=cv, scoring=make_scorer(mi_RMSE),n_jobs=-1)
Xx = ColTrans.fit_transform(X_train)
grid.fit(Xx, np.ravel(Y_train))

GridSearchCV(cv=RepeatedKFold(n_repeats=3, n_splits=10, random_state=55),
             estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'ccp_alpha': [0.0, 0.0001, 0.001, 0.01, 0.1],
                         'criterion': ['squared_error', 'absolute_error',
                                       'poisson'],
                         'max_depth': range(1, 21, 2),
                         'min_samples_split': range(2, 12, 2)},
             scoring=make_scorer(mi_RMSE))

In [87]:
mejor_modelo_RF = RandomForestRegressor(ccp_alpha=grid.best_params_.get('ccp_alpha'),
                                criterion=grid.best_params_.get('criterion'),
                                max_depth=grid.best_params_.get('max_depth'),
                                min_samples_split=grid.best_params_.get('min_samples_split'),
                                )
mejores_params_RF = grid.best_params_
pipe = Pipeline(steps=[("ct", ColTrans), ("m", mejor_modelo_RF)])
pipe.fit(X_train, Y_train)
predictions = pipe.predict(X_test)

In [86]:
## FALTA EL DIAGRAMA DE BARRAS

# **Ejercicio-11.**
Repite el ejercicio 8 y 9 para el modelo de regresión lineal múltiple para buscar sus mejores hiperparámetros (realiza la búsqueda con aquellos hiperparámetros que consideres más adecuados) y usando el conjunto de Prueba. Y realiza igualmente el análisis de importancia de factores con este modelo con un diagrama de barras


In [83]:
mi_modelo_LR = LinearRegression()

dicc_grid = {'fit_intercept': [True, False], 
            }
#================================================

grid = GridSearchCV(estimator=mi_modelo_LR, param_grid=dicc_grid, cv=cv, scoring=make_scorer(mi_RMSE),n_jobs=-1)
Xx = ColTrans.fit_transform(X_train)
grid.fit(Xx, np.ravel(Y_train))

GridSearchCV(cv=RepeatedKFold(n_repeats=3, n_splits=10, random_state=55),
             estimator=LinearRegression(), n_jobs=-1,
             param_grid={'fit_intercept': [True, False]},
             scoring=make_scorer(mi_RMSE))

In [84]:
mejor_modelo_LR = LinearRegression(fit_intercept=grid.best_params_.get('fit_intercept'))
mejores_params_LR = grid.best_params_

pipe = Pipeline(steps=[("ct", ColTrans), ("m", mejor_modelo_LR)]) # make pipeline with this model
pipe.fit(X_train, Y_train)

predictions = pipe.predict(X_test)

In [ ]:
## FALTA REALIZAR EL DAIGRAMA DE BARRAS

#**Ejercicio-12.**

In [ ]:
None

###**Fin de la Actividad de la semana 7.**